## Overview

Gemini is a family of generative AI models developed by Google DeepMind that is designed for multimodal use cases.

## Calling functions from Gemini
[`Function calling`](https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling) lets developers create a description of a function in their code, then pass that description to a language model in a request. The response from the model includes the name of a function that matches the description and the arguments to call it with.

Function calling is similar to Vertex AI Extensions in that they both generate information about functions. The difference between them is that function calling returns JSON data with the name of a function and the arguments to use in your code, whereas Vertex AI Extensions returns the function and calls it for you.

Objectives

In this tutorial, you'll build a generative AI pipeline with the Vertex AI Gemini Pro (gemini-pro) model and Python. Using your app, users can ask about exchange rates, and the system will fetch the latest data from an external API and respond to the user with the answer.

What we'll learn
* How to interact with the Gemini model using the Python client library
* How to define a function declaration and register it as a tool
* How to call Gemini and get a function call response
* How to return the function response to Gemini and respond to the user

### 1. Set Up The Environment

Install Python client library for Vertex AI if we don't

`pip install --upgrade google-cloud-aiplatform`

In [23]:
# import libraries
# import libraries
import os
import vertexai
from IPython.display import Markdown, display
from google.oauth2 import service_account
from dotenv import load_dotenv
# gemini
from vertexai.preview.generative_models import (
    FunctionDeclaration,
    GenerativeModel,
    Tool,
    Part,
    Content
)

In [2]:
# initiate service account (authentication)
json_path = '../llm-ai.json' # replace with your own service account
credentials = service_account.Credentials.from_service_account_file(json_path)

In [4]:
# start Vertex AI
load_dotenv()
vertexai.init(project=os.environ["PROJECT_ID"], # replace with your own project
              credentials=credentials)

In [5]:
# load the model
model = GenerativeModel("gemini-pro")

In [9]:
# test the model
response = model.generate_content("Why India was split?")

display(Markdown(response.text))

**The Partition of India** was the division of the British Indian Empire into two independent dominions, India and Pakistan, effective from midnight of 14 August 1947. The partition involved the geographical division of the Indian subcontinent as well as the division of its armed forces, civil services, and other resources.

The partition was the culmination of a long history of political and communal tensions between Hindus and Muslims in India. The Indian National Congress, which represented the Hindu majority, demanded independence from British rule and the creation of a united India. The Muslim League, which represented the Muslim minority, demanded the creation of a separate Muslim state in India.

In 1946, the British government announced its plan to divide India into two dominions, India and Pakistan. The plan was accepted by the Indian National Congress but was rejected by the Muslim League. The Muslim League launched a direct action campaign, which resulted in communal violence and riots in Calcutta and other cities.

In 1947, the British government decided to grant independence to India and Pakistan on the condition that the two dominions would agree to a partition plan. The partition plan was accepted by both the Indian National Congress and the Muslim League.

The partition of India was a traumatic event that resulted in the displacement of millions of people and the deaths of hundreds of thousands. The partition also led to the creation of two separate states, India and Pakistan, which have been in conflict with each other ever since.

**Some of the key factors that contributed to the partition of India include:**

* **The rise of nationalism:** The early 20th century saw the rise of nationalism in both India and Pakistan. This led to increasing demands for independence from British rule.
* **Communal tensions:** There were long-standing tensions between Hindus and Muslims in India. These tensions were exacerbated by the rise of nationalism and the competing demands of the Indian National Congress and the Muslim League.
* **The British government's divide-and-rule policy:** The British government often played one religious group against the other in order to maintain its control over India. This policy contributed to the growth of communal tensions.
* **The role of the Indian National Congress and the Muslim League:** The Indian National Congress represented the Hindu majority, while the Muslim League represented the Muslim minority. The two parties were unable to agree on a power-sharing arrangement, which led to deadlock and ultimately partition.

**The partition of India was a tragedy that had a profound impact on the history of the Indian subcontinent. It led to the displacement of millions of people, the deaths of hundreds of thousands, and the creation of two separate states that have been in conflict with each other ever since.**

### 2. Understand The Problem

Have you ever interacted with a large language model or generative AI model and asked it about real-time or current information, only to get a response with outdated information or inaccurate information?

Let's try it now!

In [10]:
response = model.generate_content(
    "What's the exchange rate for rupiahs to dollars today?"
)
display(Markdown(response.text))

I do not have real-time information and my knowledge cutoff is April 2023. Therefore, I cannot provide you with the current exchange rate for rupiahs to dollars. You may want to check a reputable financial news source or a currency exchange website for the most up-to-date information.

If an end-user received this type of response, they would need to switch contexts to look up the currencies that they're interested in, fetch the latest exchange rate, and perform any conversions on their own.

Ideally, a generative model pipeline could handle some or all of these tasks for the user. In the next part, we'll try some common workarounds for getting structured responses from generative models so that we can call external systems.

### 3. Try common workarounds

When working with generative models in scenarios where we need up-to-date information or data from external sources, we could call an external API then feed the results back to the generative model for it to use in its response.

Before we call an external system, we need to determine the right function to use, extract the relevant parameters from the user, and put the parameters into a structured data object. This usually involves exhaustive prompt engineering to coerce the generative model to output valid structured data.

Let's revisit the question that we asked in the previous section and add some additional instructions for the model. Try sending the following request to the Gemini model:

In [11]:
user_prompt = "What's the exchange rate from rupiahs to US dollars today?"

response = model.generate_content(f"""
Your task is to extract parameters from the user's input and return it as a
structured JSON payload. The user will ask about the exchange rate and which
currency they are converting from and converting to.

User input: {user_prompt}

Please extract the currencies as parameters and put them in a JSON object.
"""
)

display(Markdown(response.text))

```json
{
    "from": "rupiahs",
    "to": "US dollars"
}
```

In [12]:
# print it
print(response.text)

```json
{
    "from": "rupiahs",
    "to": "US dollars"
}
```


In particular, the first and last lines of the text response include backticks to delimit the code block, the first line includes a language specifier, and the values in the JSON object are not the standard three-letter currency abbreviations that a currency exchange API would expect as input parameters.

We could try to use Python to post-process this text into valid JSON and a dictionary, add more instructions to the prompt, provide one or more examples of desired output, fine-tune the model, or make another call to the generative model asking it to clean up the JSON.

But there is a more deterministic way! Let's learn how to use function calling in Gemini to query for information in external services and return relevant responses to end-users.

### 4. How function calling works

Before we get started with parameter extraction and function calling, let's walk through the steps of function calling and which components are used along the way.

![Function Calling](../assets/gemini-function-calling.png)

#### A. Define a function and tool
First, we'll define a function declaration and a tool so that the model knows which functions it can call and how to call them. This step involves describing one or more functions for the model to use, the relevant input parameters and data types, and which parameters are required.

```
API Specification → Function Declaration → Tool
```

#### B. Generate a function call
Then, we'll send a request to the model with a prompt so that it can generate a function call with the appropriate function name and parameters. This step involves sending a prompt and list of tools to the model, and receiving a response with a function name and parameters to use.

```
Model → Tool → Function call with parameters
```

#### C. Make an API request
Then, we'll use the function and parameters to make a an API request so that you can retrieve the latest information from an external system. This step involves setting up the input parameters, making an API request, and receiving a response object from the API.

```
Function call with parameters → API → Response
```

#### D. Return a summary
Finally, we'll pass the function response back to the model so that it can generate a response to the end-user's initial prompt. This step involves returning the function response along with the previous conversation turns to the model, then handling the final response.

```
Function Response → Model → Response to user
```


### 5. Choose our API

Now that we understand the overall flow and specific steps in function calling, we'll build a generative AI pipeline to fetch the latest currency exchange rates. First, we'll need to select which API that we want to use as a source of information.

For our currency exchange app, we'll use the REST API at [https://www.frankfurter.app/](https://www.frankfurter.app/) to fetch the latest information about global exchange rates.

To interact with this REST API, we might make a REST API call with `requests` in Python as:

In [13]:
import requests
url = "https://api.frankfurter.app/latest"
response = requests.get(url)
response.text

'{"amount":1.0,"base":"EUR","date":"2024-01-26","rates":{"AUD":1.6483,"BGN":1.9558,"BRL":5.3366,"CAD":1.4607,"CHF":0.9396,"CNY":7.7994,"CZK":24.748,"DKK":7.4549,"GBP":0.85368,"HKD":8.4933,"HUF":386.95,"IDR":17132,"ILS":4.0057,"INR":90.37,"ISK":148.3,"JPY":160.62,"KRW":1451.08,"MXN":18.6582,"MYR":5.1393,"NOK":11.3325,"NZD":1.7801,"PHP":61.28,"PLN":4.3775,"RON":4.9765,"SEK":11.3203,"SGD":1.457,"THB":38.679,"TRY":32.944,"USD":1.0871,"ZAR":20.426}}'

Because function calling in Gemini does not actually make the external API call for us, there are no such restrictions on what type of API that we use! We could use a Cloud Run Service, a Cloud Function, an API request to a Google Cloud service, or any external REST API.


Referring back to the REST API at [https://www.frankfurter.app/docs/](www.frankfurter.app/docs/), we can see that it accepts the following input parameters:

| Parameter | Type   | Description                        |
|-----------|--------|------------------------------------|
| _from_      | String | Currency to convert from           |
| _to_        | String | Currency to convert to             |
| _date_      | String | Date to fetch the exchange rate for|

Now, let's register this as a `FunctionDeclaration` using the Python SDK for Gemini:


In [18]:
get_exchange_rate_func = FunctionDeclaration(
    name="get_exchange_rate",
    description="Get the exchange rate for currencies between countries",
    parameters={
    "type": "object",
    "properties": {
        "currency_date": {
            "type": "string",
            "description": "A date that must always be in YYYY-MM-DD format or the value 'latest' if a time period is not specified"
        },
        "currency_from": {
            "type": "string",
            "description": "The currency to convert from in ISO 4217 format"
        },
        "currency_to": {
            "type": "string",
            "description": "The currency to convert to in ISO 4217 format"
        }
    },
         "required": [
            "currency_from",
            "currency_date",
      ]
  },
)

Be sure to use as much detail as possible in the function and parameter descriptions since the generative model will use this information to determine which function to select and how to fill the parameters in the function call.

Finally, you'll define a `Tool` that includes the function declaration:

In [19]:
exchange_rate_tool = Tool(
    function_declarations=[get_exchange_rate_func],
)

Here, we're using one function declaration within a tool, but note that we can **register one or more** function declarations in a tool, and the model will select the appropriate function to use at runtime.

We've completed the configuration of our function and tool definitions. In the next section, we'll call the generative model with this tool and get back a function call that we can use to call the REST API.


### 6. Generate a function call

Now we can prompt the generative model and include the tool that we defined:

In [20]:
prompt = """What is the exchange rate from to Korean Won to Indonesian Rupiah?
How much is 5000 Korean wons worth in Indonesian Rupiahs?"""

response = model.generate_content(
    prompt,
    tools=[exchange_rate_tool],
)

print(response.candidates[0].content)

role: "model"
parts {
  function_call {
    name: "get_exchange_rate"
    args {
      fields {
        key: "currency_to"
        value {
          string_value: "IDR"
        }
      }
      fields {
        key: "currency_from"
        value {
          string_value: "KRW"
        }
      }
      fields {
        key: "currency_date"
        value {
          string_value: "latest"
        }
      }
    }
  }
}



It looks like the model selected the one available function and returned a function call for the `get_exchange_rate` function along with the parameters. And the parameters are in the correct format that we wanted. Hooray for getting structured responses from generative models!

In the next part, we'll use the information in the response to make an API request.

### 7. Make an API request

Recall that function calling in Gemini does not actually make the external API call for you. Rather, we are free to use any language, library, or framework that we'd like!

Here we'll use the `requests` library in Python to call the exchange rate REST API.

Let's unpack the response into a Python dictionary:

In [21]:
params = {}
for key, value in response.candidates[0].content.parts[0].function_call.args.items():
    params[key[9:]] = value
params

{'from': 'KRW', 'date': 'latest', 'to': 'IDR'}

Now we can call requests or any other method:

In [22]:
url = f"https://api.frankfurter.app/{params['date']}"
api_response = requests.get(url, params=params)
api_response.text

'{"amount":1.0,"base":"KRW","date":"2024-01-26","rates":{"IDR":11.8064}}'

And we have our response from the REST API, with the latest exchange rate information from today. In the next section, we'll pass this information back to the model so that it can generate a relevant response for the user.


### 8. Generate a response

Finally, let's generate a response for the user by passing back the function response to the model in the next conversation turn:

In [24]:
# user prompt
print(prompt)

What is the exchange rate from to Korean Won to Indonesian Rupiah?
How much is 5000 Korean wons worth in Indonesian Rupiahs?


In [25]:
response = model.generate_content(
    [
    Content(role="user", parts=[
        Part.from_text(prompt + """Give your answer in steps with lots of detail
            and context, including the exchange rate and date."""),
    ]),
    Content(role="function", parts=[
        Part.from_dict({
            "function_call": {
                "name": "get_exchange_rate",
            }
        })
    ]),
    Content(role="function", parts=[
        Part.from_function_response(
            name="get_exchange_rate",
            response={
                "content": api_response.text,
            }
        )
    ]),
    ],
    tools=[exchange_rate_tool],
)


response.candidates[0].content.parts[0].text

' 1 Korean Won is equal to 11.8064 Indonesian Rupiah in 2024-01-26. \n5000 Korean Won is equal to 59032 Indonesian Rupiah in 2024-01-26.'

In [26]:
# beautify the response
display(Markdown(response.candidates[0].content.parts[0].text))

 1 Korean Won is equal to 11.8064 Indonesian Rupiah in 2024-01-26. 
5000 Korean Won is equal to 59032 Indonesian Rupiah in 2024-01-26.